In [2]:
# Constants for this project
import constants
from constants import *

import auxFunctions as af

import importlib

import os
import numpy as np
import pandas as pd

import papermill as pm
import shutil
shutil.copy('constants.py', 'export/')
shutil.copy('auxFunctions.py', 'export/')

# Change the current working directory
os.chdir("/home/yonatan/Documents/projects/vaccines/code")

In [3]:
af.print_kernel_file()

/home/yonatan/.local/share/jupyter/runtime/kernel-v2-106959wx5pYn8gP8Bz.json


In [4]:
importlib.reload(af)

datasets, metadata = af.load_data()
age_str = f", subjects over the age of {age_threshlod}" if bOlderOnly else ""
print(f"""Analysis parameters: discarding serprotected subjects: {bDiscardSeroprotected}{age_str}""")

# Turn on debug here for running a single dataset through analyze_dataset()
bDebug = False
if bDebug:
    results = af.debug_single_dataset(datasets, metadata)
else:
    results = af.analyze_all_datasets(datasets, metadata)

Analysis parameters: discarding serprotected subjects: True
               uid  IMMAGE   Age   D0     D28  FC  Cluster  adjFC
0   SUB187451.1289    0.08 26.00 0.00  320.00 inf        0    NaN
1   SUB187452.1289   -0.18 23.00 0.00  640.00 inf        0    NaN
2   SUB187453.1289   -0.06 28.00 0.00  640.00 inf        0    NaN
3   SUB187454.1289    0.15 28.00 0.00 1280.00 inf        0    NaN
4   SUB187455.1289   -0.25 25.00 0.00  640.00 inf        0    NaN
5   SUB187456.1289    0.28 43.00 0.00  160.00 inf        0    NaN
6   SUB187457.1289   -0.20 31.00 0.00  640.00 inf        0    NaN
7   SUB187458.1289   -0.28 22.00 0.00  160.00 inf        0    NaN
8   SUB187460.1289    0.07 26.00 0.00  320.00 inf        0    NaN
9   SUB187461.1289   -0.17 23.00 0.00 1280.00 inf        0    NaN
10  SUB187462.1289   -0.01 34.00 0.00  640.00 inf        0    NaN
11  SUB187463.1289    0.00 27.00 0.00 2560.00 inf        0    NaN
12  SUB187464.1289    0.09 25.00 0.00 1280.00 inf        0    NaN
13  SUB187465.12

/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/base.py:1152: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/base.py:1152: ConvergenceWarning: Number of distinct clusters (1

In [5]:
# Get all the analyses that look somewhat promising based on the composite (F1 and over-threshold rate) metric
cols_to_access = [
    dataset_col,
    strain_col,
    day_col,
    "max_difference",
    "bAdjustMFC",
]

# Dynamically add all sub-columns for 'Composite'
composite_columns = [col for col in results.columns if col[0] == 'Composite']
cols_to_access.extend(composite_columns)
# results["max_difference"] = results.apply(lambda row: max(row["Composite", "IMMAGE"] - row["Composite", "Age"], row["Composite", "Multivariate"] - row["Composite", "Age"]), axis=1)
results["max_difference"] = results.apply(lambda row: max(row["Score", "IMMAGE"] - row["Score", "Age"], row["Score", "Multivariate"] - row["Score", "Age"]), axis=1)
results = results.sort_values(by="max_difference", ascending=False)
# results = results.loc[results["max_difference"] > 0.1]

# Save results to a pickle file, to be used in other places
results.to_pickle(f"pickles/results-max-diff{ROC_PRC_str}.pkl")

In [6]:
%%script true
Generate figures as html for promising datasets
for index, row in results.iterrows():
    parameters = {
        "bAdjustMFC" : row["bAdjustMFC"],
        "dataset_name": row[dataset_col],
        "strain_index": row[strain_index_col],
        "day": row[day_col]
    }
    adjFC_str = f"_adjFC" if row["bAdjustMFC"] else ""
    print(f'exporting {row[dataset_col]}, strain no. {row[strain_index_col]}: {row[strain_col]}, day: {row[day_col]}')
    output_notebook_name = f"{row[dataset_col]}_{row[strain_col]}_{row[day_col]}{seroprotected_str}{age_restrict_str}{adjFC_str}"
    output_notebook = f"export/{output_notebook_name}.ipynb"
    try:
            pm.execute_notebook(
                    input_path="vaccines-4-for-papermill.ipynb",
                    output_path=output_notebook,
                    parameters=parameters,
                    prepare_only=True,
            )
    except Exception as e:
            print (f"******\nCaught exception when runnnig {output_notebook}\n******\n")
            raise(e)
    # Export the executed notebook to HTML
    output_html = f"{output_notebook_name}.html"
    os.system(f"jupyter nbconvert --execute --no-input --to html {output_notebook} --output {output_html}")

In [7]:
# Save promising results to a spreadsheet
def generate_html_path(row):
    adjFC_str = f"_adjFC" if row["bAdjustMFC"] else ""
    output_html = f"{row[dataset_col]}_{row[strain_col]}_{row[day_col]}{seroprotected_str}{age_restrict_str}{adjFC_str}.html"
    output_html = os.path.join(af.get_dir_by_name('code'), "export", output_html)
    output_html = '=HYPERLINK("file://' + output_html + '", "Link")'
    return output_html

results = results[cols_to_access]
results['html_path'] = results.apply(generate_html_path, axis=1)
results.to_csv(f"export/results_{seroprotected_str}{age_restrict_str}{adjFC_str}.csv", index=False, float_format='%.2f')

In [8]:
results

,Dataset,Strain,Day,max_difference,bAdjustMFC,"(Composite, IMMAGE)","(Composite, Age)","(Composite, Multivariate)",html_path
21,GSE52245.SDY1260,IgG1_serotype_A,D30,0.35,True,0.35,0.24,0.52,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/GSE52245.SDY1260_IgG1_serotype_A_D30_discard_seroprotected_adjFC.html"", ""Link"")"
43,GSE59654.SDY180,A_Brisbane_10_2007,HAI.D28,0.27,True,0.42,0.31,0.67,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/GSE59654.SDY180_A_Brisbane_10_2007_HAI.D28_discard_seroprotected_adjFC.html"", ""Link"")"
36,GSE52245.SDY1260,IgM_serotype_C,D30,0.25,False,0.44,0.22,0.44,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/GSE52245.SDY1260_IgM_serotype_C_D30_discard_seroprotected.html"", ""Link"")"
32,GSE52245.SDY1260,IgG_serotype_C,D30,0.25,False,0.52,0.22,0.43,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/GSE52245.SDY1260_IgG_serotype_C_D30_discard_seroprotected.html"", ""Link"")"
16,GSE48023.SDY1276,A_Brisbane_10_2007,HAI.D14,0.25,False,0.44,0.24,0.44,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/GSE48023.SDY1276_A_Brisbane_10_2007_HAI.D14_discard_seroprotected.html"", ""Link"")"
...,...,...,...,...,...,...,...,...,...
29,GSE52245.SDY1260,IgG_serotype_A,D30,-0.02,True,0.23,0.39,0.60,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/GSE52245.SDY1260_IgG_serotype_A_D30_discard_seroprotected_adjFC.html"", ""Link"")"
65,GSE59743.SDY400,A_Victoria_361_2011,HAI.D28,-0.05,True,0.65,0.65,0.66,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/GSE59743.SDY400_A_Victoria_361_2011_HAI.D28_discard_seroprotected_adjFC.html"", ""Link"")"
74,SDY296,A_California_7_2009,D28.HAI,-0.05,False,0.22,0.60,0.55,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/SDY296_A_California_7_2009_D28.HAI_discard_seroprotected.html"", ""Link"")"
58,GSE59654.SDY520,A_Victoria_361_2011,HAI.D28,-0.06,False,0.57,0.39,0.45,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/GSE59654.SDY520_A_Victoria_361_2011_HAI.D28_discard_seroprotected.html"", ""Link"")"
